In [1]:
# Main notebook for investigating the parametersspace of vaccine vs cumulative incidence, from kommune-data
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

from matplotlib.colors import LinearSegmentedColormap

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [2]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 


In [3]:
list(os.walk(path_data))[-1][0]

'c:\\Users\\rakrpe\\Documents\\PandemiX\\GithubRepos\\PandemiX\\notebooks\\..\\DanskeData\\ssi_data\\SSI_data_2021-11-08'

In [4]:
# ssidatapath = "ssi_data"
# rootdir = os.getcwd() +"\\" + ssidatapath


# for subdir, dirs, files in os.walk(path_data):
#     if not len(files) == 0:
#         latestdir = subdir
#         latestDate = pd.to_datetime(subdir[-10:])
#         # curdf = pd.read_csv(latestdir+'\\Cases_by_age.csv',delimiter = ';',dtype=str)
         
latestdir = list(os.walk(path_data))[-1][0]      
latestDate = pd.to_datetime(latestdir[-10:])  
         
latestDash = rootdir_data + f'\\ssi_dashboard\\SSI_dashboard_{latestDate:%Y-%m-%d}\\'
latestVacc = rootdir_data + f'\\ssi_vacc\\SSI_vacc_{latestDate:%Y-%m-%d}\\'
kortdf = pd.read_csv(latestDash+'Kommunalt_DB\\11_kort_pr_kommune.csv',delimiter = ';',dtype=str,encoding='latin1')
countdf = pd.read_csv(latestDash+'Kommunalt_DB\\17_tilfaelde_fnkt_alder_kommuner.csv',delimiter = ';',dtype=str,encoding='latin1')

vaccdf = pd.read_csv(latestVacc+'Vaccine_DB\\Vaccinationsdaekning_kommune.csv',delimiter = ';',dtype=str,encoding='latin1')


kortdf['Bekræftede tilfælde de seneste 7 dage'] = pd.to_numeric(kortdf['Bekræftede tilfælde de seneste 7 dage'])
kortdf['Testede de seneste 7 dage'] = pd.to_numeric(kortdf['Testede de seneste 7 dage'])
kortdf['Testede de seneste 7 dage'] = pd.to_numeric(kortdf['Testede de seneste 7 dage'])
kortdf['Positivprocent 7 dage'] = pd.to_numeric(kortdf['Positivprocent 7 dage'].astype(str).apply(lambda x: x.replace(',','.')))

countdf['Bekræftede tilfælde'] = pd.to_numeric(countdf['Bekræftede tilfælde'])

vaccdf['Antal faerdigvacc.'] = pd.to_numeric(vaccdf['Antal faerdigvacc.'])
vaccdf['Vacc.dækning faerdigvacc. (%)'] = pd.to_numeric(vaccdf['Vacc.dækning faerdigvacc. (%)'])
vaccdf['Antal borgere'] = pd.to_numeric(vaccdf['Antal borgere'])

# latestdir
# subdir
# curdf

In [5]:
# vaccdf

In [6]:
# df_kommune = pd.DataFrame()

komToShow = kortdf.Kommunenavn.unique()

caseSums = countdf.groupby('Kommune')['Bekræftede tilfælde'].sum()

# counter = 0
rows_list = []
for ko in komToShow:
    
    curDict = {}
    
    try:
        slice_kortdf = kortdf[kortdf.Kommunenavn == ko].iloc[0]
        
        slice_vaccdf = vaccdf[vaccdf.Kommunenavn == ko].iloc[0]
    except:
        continue
    
    curCode = slice_kortdf.Kommune
    curSum = caseSums[curCode]
    curBorger = slice_vaccdf['Antal borgere']
    
    curDict.update(Kommunekode=slice_kortdf.Kommune)
    curDict.update(Kommunenavn=ko)
    curDict.update(Indbyggere=curBorger)
    curDict.update(Faerdigvacc=int(slice_vaccdf['Antal faerdigvacc.']))
    
    ratioDone = slice_vaccdf['Antal faerdigvacc.']/slice_vaccdf['Antal borgere']
    curDict.update(FaerdigvaccRatio=ratioDone)
    
    ratioCase = curSum/slice_vaccdf['Antal borgere']
    curDict.update(CasesSum=curSum)
    curDict.update(CasesSumPerBorger=ratioCase)
    
    
    ratioCase7 = slice_kortdf['Bekræftede tilfælde de seneste 7 dage']/slice_vaccdf['Antal borgere']
    curDict.update(Cases7=slice_kortdf['Bekræftede tilfælde de seneste 7 dage'])
    curDict.update(Cases7PerBorger=ratioCase7)
    
    curDict.update(Test7=slice_kortdf['Testede de seneste 7 dage'])
    curDict.update(PosPct7=slice_kortdf['Positivprocent 7 dage'])
    
    
    rows_list.append(curDict)
    
df_kommune = pd.DataFrame(rows_list)
df_kommune.head(5)
    
    


,Kommunekode,Kommunenavn,Indbyggere,Faerdigvacc,FaerdigvaccRatio,CasesSum,CasesSumPerBorger,Cases7,Cases7PerBorger,Test7,PosPct7
0,101,København,644538,461747,0.716400,75805,0.117611,2931,0.004547,76688,3.82
1,147,Frederiksberg,103838,79853,0.769015,11325,0.109064,434,0.004180,11491,3.78
2,151,Ballerup,49175,35658,0.725125,5170,0.105135,217,0.004413,6793,3.19
3,153,Brøndby,35544,23496,0.661040,5187,0.145932,249,0.007005,4982,5.00
4,155,Dragør,14618,11128,0.761253,1281,0.087632,76,0.005199,2469,3.08


In [7]:
# # kortdf[kortdf.Kommunenavn == ko].iloc[0]
# # kortdf
# vaccdf[vaccdf.Kommunenavn == ko]
# vaccdf.head(5)
# countdf.groupby('Kommune')['Bekræftede tilfælde'].sum()
# countdf

fig,ax1 = plt.subplots(figsize=(12,8))

ax1.plot(100*df_kommune.FaerdigvaccRatio,100*df_kommune.CasesSum/df_kommune.Indbyggere,'k.')
# ax1.set_xlim([0,1])
# ax1.set_ylim([0,1])
# ax1.set_xlim([0.6,0.85])
# ax1.set_ylim([0,0.25])
ax1.set_xlim([60,85])
ax1.set_ylim([0,25])
for k in range(len(df_kommune)):
    ax1.text(100*df_kommune.FaerdigvaccRatio[k],100*df_kommune.CasesSum[k]/df_kommune.Indbyggere[k],df_kommune.Kommunenavn[k])
    
ax1.set_xlabel('Vaccinationsrate [%]')
ax1.set_ylabel('Kommuleret incidens [%]')


plt.savefig(path_figs+'Kommune_VaccVsCumuInci')


fig,ax1 = plt.subplots(figsize=(12,8))

# ax1.plot(100*df_kommune.FaerdigvaccRatio,df_kommune.Cases7PerBorger,'k.')
ax1.plot(100*df_kommune.FaerdigvaccRatio,100000*df_kommune.Cases7PerBorger,'k.')
for k in range(len(df_kommune)):
    # ax1.text(100*df_kommune.FaerdigvaccRatio[k],df_kommune.Cases7PerBorger[k],df_kommune.Kommunenavn[k])
    ax1.text(100*df_kommune.FaerdigvaccRatio[k],100000*df_kommune.Cases7PerBorger[k],df_kommune.Kommunenavn[k])
    
ax1.set_xlabel('Vaccinationsrate [%]')
# ax1.set_ylabel('Bekræftede tilfælde de sidste 7 dage, per borger')
ax1.set_ylabel('Incidens per 100.000 borgere, seneste 7 dage')

plt.tight_layout()

plt.savefig(path_figs+'Kommune_VaccVsInci')




fig,ax1 = plt.subplots(figsize=(12,8))

# ax1.plot(100*df_kommune.FaerdigvaccRatio,df_kommune.Cases7PerBorger,'k.')
ax1.plot(100*(df_kommune.CasesSum/df_kommune.Indbyggere+df_kommune.FaerdigvaccRatio),100000*df_kommune.Cases7PerBorger,'k.')
for k in range(len(df_kommune)):
    # ax1.text(100*df_kommune.FaerdigvaccRatio[k],df_kommune.Cases7PerBorger[k],df_kommune.Kommunenavn[k])
    ax1.text(100*(df_kommune.CasesSum[k]/df_kommune.Indbyggere[k]+df_kommune.FaerdigvaccRatio[k]),100000*df_kommune.Cases7PerBorger[k],df_kommune.Kommunenavn[k])
    
ax1.set_xlabel('Immunitetsrate [%]')
# ax1.set_ylabel('Bekræftede tilfælde de sidste 7 dage, per borger')
ax1.set_ylabel('Incidens per 100.000 borgere, seneste 7 dage')

plt.tight_layout()

plt.savefig(path_figs+'Kommune_VaccAndCumuVsInci')


fig,ax1 = plt.subplots(figsize=(12,8))

xx = df_kommune.CasesSum/df_kommune.Indbyggere
yy = df_kommune.FaerdigvaccRatio
ax1.plot(100*(xx+yy - xx*yy),100000*df_kommune.Cases7PerBorger,'k.')
for k in range(len(df_kommune)):
    # ax1.text(100*df_kommune.FaerdigvaccRatio[k],df_kommune.Cases7PerBorger[k],df_kommune.Kommunenavn[k])
    ax1.text(100*(xx[k]+yy[k] - xx[k]*yy[k]),100000*df_kommune.Cases7PerBorger[k],df_kommune.Kommunenavn[k])
    
ax1.set_xlabel('Immunitetsrate, korrigeret [%]')
# ax1.set_ylabel('Bekræftede tilfælde de sidste 7 dage, per borger')
ax1.set_ylabel('Incidens per 100.000 borgere, seneste 7 dage')

plt.tight_layout()

plt.savefig(path_figs+'Kommune_VaccAndCumuVsInci_Corrected')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
fig,ax1 = plt.subplots(figsize=(8,8))

minX = 0
maxX = 1
minY = 0
maxY = 1

minX = 0.6
maxX = 0.9
minY = 0
maxY = 0.3


R0 = 4.7
# R0 = 6.0

S = lambda V, Preval : (1-V)*(1-Preval)
Re = lambda R0, V, Preval: R0*S(V, Preval)
V = lambda Re, R0, Preval: 1 - (Re/R0)*(1/(1-Preval))
P = lambda Re, R0, Vacc: 1 - (Re/R0)*(1/(1-Vacc))

# Adjust here, for the visible range in plot:
resolution = 100
X = vacc_list = np.linspace(minX, maxX, resolution)
Y = preval_list = np.linspace(minY,maxY, resolution)
Z = np.array([[Re(R0, v, p) for v in vacc_list] for p in preval_list]) # Y=preval, X=vacc



# r0 = 4.7
# res = np.linspace(0.05,0.95,numAreas)*r0
res = np.arange(0,R0,0.25)
numAreas = len(res)

cmap = LinearSegmentedColormap.from_list('unnamed', 
                                                    [
                                                        (0,1,0),
                                                        (1,0.9,0.7),
                                                        (1,0,0),
                                                        (1,0,0),
                                                        (1,0,0),
                                                        (1,0,0),
                                                    ] 
                                                    , N=numAreas)

#ax.pcolor(X, Y, Z)
# ax.contourf(X, Y, Z, 100, cmap="viridis")
# ax.contourf(X, Y, Z, numAreas, cmap=cmap)
ax1.contourf(X, Y, Z, levels=res,cmap=cmap)


for i, re in enumerate([1]):
    IsoVac = np.linspace(0,1-(re/R0),100)
    ax1.plot(IsoVac, (lambda V: P(re, R0, V))(IsoVac), linewidth = 10,alpha=0.5,zorder=5, color="grey", label=int(i==0)*"$R_{e} = 1$-Isocline")

CS = ax1.contour(X, Y, Z, levels=res, zorder=6,colors='k') 
# clab_pos = [(1-np.sqrt(y),1-np.sqrt(y)) for y in res/R0]
# clab_pos = [(1-np.sqrt(y),1-np.sqrt(y)) for y in res/R0]
numLabelsToShow = 10
clab_pos = list(zip(np.linspace(minX,maxX,numLabelsToShow),np.linspace(minY,maxY,numLabelsToShow)))
ax1.clabel(CS,fmt='%.2f', inline=True, fontsize=10,manual=clab_pos)
# ax.clabel(CS,fmt='%.2f', inline=True, fontsize=10)
    

# # ax1.plot(ratioVacc,numInf/popSize,'*',zorder=19)
# # ax1.plot(ratioVacc,2*numInf/popSize,'*r',zorder=19)
# for k in range(len(ratioVacc)):
#     # ax1.plot(ratioVacc[k],numInf[k]/popSize[k],'*',zorder=19)
#     ax1.plot([ratioVacc[k],ratioVacc[k]],[numInf[k]/popSize[k],2*numInf[k]/popSize[k]],'.-',zorder=19)
#     ax1.text(ratioVacc[k],numInf[k]/popSize[k],komNames[k],zorder=20)
    

for k in range(len(df_kommune)):
    # ax1.plot(100*df_kommune.FaerdigvaccRatio[k],100*df_kommune.CasesSum[k]/df_kommune.Indbyggere[k],'k.')
    # ax1.text(100*df_kommune.FaerdigvaccRatio[k],100*df_kommune.CasesSum[k]/df_kommune.Indbyggere[k],df_kommune.Kommunenavn[k])
    ax1.plot(df_kommune.FaerdigvaccRatio[k],df_kommune.CasesSum[k]/df_kommune.Indbyggere[k],'k.')
    ax1.text(df_kommune.FaerdigvaccRatio[k],df_kommune.CasesSum[k]/df_kommune.Indbyggere[k],df_kommune.Kommunenavn[k])
    
# ax.axis('equal')
ax1.set_xlim([minX,maxX])
ax1.set_ylim([minY,maxY])

# plt.savefig('figs/KommuneFigur')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 0.3)

In [9]:
fig,ax1 = plt.subplots(figsize=(8,8))

minX = 0
maxX = 1
minY = 0
maxY = 1

# minX = 0.6
# maxX = 0.9
# minY = 0
# maxY = 0.3


R0 = 4.7
# R0 = 6.0

S = lambda V, Preval : (1-V)*(1-Preval)
S = lambda V, Preval : (1-V-Preval)
Re = lambda R0, V, Preval: R0*S(V, Preval)
V = lambda Re, R0, Preval: 1 - (Re/R0)*(1/(1-Preval))
P = lambda Re, R0, Vacc: 1 - (Re/R0)*(1/(1-Vacc))

# Adjust here, for the visible range in plot:
resolution = 100
X = vacc_list = np.linspace(minX, maxX, resolution)
Y = preval_list = np.linspace(minY,maxY, resolution)
Z = np.array([[Re(R0, v, p) for v in vacc_list] for p in preval_list]) # Y=preval, X=vacc



# r0 = 4.7
# res = np.linspace(0.05,0.95,numAreas)*r0
res = np.arange(0,R0+0.25,0.25)
numAreas = len(res)

cmap = LinearSegmentedColormap.from_list('unnamed', 
                                                    [
                                                        (0,1,0),
                                                        (1,0.9,0.7),
                                                        (1,0,0),
                                                        (1,0,0),
                                                        (1,0,0),
                                                        (1,0,0),
                                                    ] 
                                                    , N=numAreas)

ax1.contourf(X, Y, Z, levels=res,cmap=cmap)


# for i, re in enumerate([1]):
#     IsoVac = np.linspace(0,1-(re/R0),100)
#     ax1.plot(IsoVac, (lambda V: P(re, R0, V))(IsoVac), linewidth = 10,alpha=0.5,zorder=5, color="grey", label=int(i==0)*"$R_{e} = 1$-Isocline")

CS = ax1.contour(X, Y, Z, levels=res, zorder=6,colors='k') 
# clab_pos = [(1-np.sqrt(y),1-np.sqrt(y)) for y in res/R0]
clab_pos = [(1-np.sqrt(y),1-np.sqrt(y)) for y in res/R0]
# numLabelsToShow = 10
# clab_pos = list(zip(np.linspace(minX,maxX,numLabelsToShow),np.linspace(minY,maxY,numLabelsToShow)))
ax1.clabel(CS,fmt='%.2f', inline=True, fontsize=10,manual=clab_pos)
# ax.clabel(CS,fmt='%.2f', inline=True, fontsize=10)
    

ax1.set_xlim([minX,maxX])
ax1.set_ylim([minY,maxY])

plt.savefig(path_figs+'Vacc_Inci_Pars_Dependent')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
fig,ax1 = plt.subplots(figsize=(8,8))

minX = 0
maxX = 1
minY = 0
maxY = 1

# minX = 0.6
# maxX = 0.9
# minY = 0
# maxY = 0.3


R0 = 4.7
# R0 = 6.0

S = lambda V, Preval : (1-V)*(1-Preval)
# S = lambda V, Preval : (1-V-Preval)
Re = lambda R0, V, Preval: R0*S(V, Preval)
V = lambda Re, R0, Preval: 1 - (Re/R0)*(1/(1-Preval))
P = lambda Re, R0, Vacc: 1 - (Re/R0)*(1/(1-Vacc))

# Adjust here, for the visible range in plot:
resolution = 100
X = vacc_list = np.linspace(minX, maxX, resolution)
Y = preval_list = np.linspace(minY,maxY, resolution)
Z = np.array([[Re(R0, v, p) for v in vacc_list] for p in preval_list]) # Y=preval, X=vacc



# r0 = 4.7
# res = np.linspace(0.05,0.95,numAreas)*r0
res = np.arange(0,R0+0.25,0.25)
numAreas = len(res)

cmap = LinearSegmentedColormap.from_list('unnamed', 
                                                    [
                                                        (0,1,0),
                                                        (1,0.9,0.7),
                                                        (1,0,0),
                                                        (1,0,0),
                                                        (1,0,0),
                                                        (1,0,0),
                                                    ] 
                                                    , N=numAreas)

ax1.contourf(X, Y, Z, levels=res,cmap=cmap)


# for i, re in enumerate([1]):
#     IsoVac = np.linspace(0,1-(re/R0),100)
#     ax1.plot(IsoVac, (lambda V: P(re, R0, V))(IsoVac), linewidth = 10,alpha=0.5,zorder=5, color="grey", label=int(i==0)*"$R_{e} = 1$-Isocline")

CS = ax1.contour(X, Y, Z, levels=res, zorder=6,colors='k') 
# clab_pos = [(1-np.sqrt(y),1-np.sqrt(y)) for y in res/R0]
clab_pos = [(1-np.sqrt(y),1-np.sqrt(y)) for y in res/R0]
# numLabelsToShow = 10
# clab_pos = list(zip(np.linspace(minX,maxX,numLabelsToShow),np.linspace(minY,maxY,numLabelsToShow)))
ax1.clabel(CS,fmt='%.2f', inline=True, fontsize=10,manual=clab_pos)
# ax.clabel(CS,fmt='%.2f', inline=True, fontsize=10)
    

ax1.set_xlim([minX,maxX])
ax1.set_ylim([minY,maxY])

plt.savefig(path_figs+'Vacc_Inci_Pars_Independent')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
fig,ax1 = plt.subplots(figsize=(8,8))

minX = 0
maxX = 1
minY = 0
maxY = 1

# minX = 0.6
# maxX = 0.9
# minY = 0
# maxY = 0.3


R0 = 4.7
# R0 = 6.0

# S = lambda V, Preval : (1-V)*(1-Preval)
S = lambda V, Preval : (1-V-Preval + 0.5 * V * Preval)
Re = lambda R0, V, Preval: R0*S(V, Preval)
V = lambda Re, R0, Preval: 1 - (Re/R0)*(1/(1-Preval))
P = lambda Re, R0, Vacc: 1 - (Re/R0)*(1/(1-Vacc))

# Adjust here, for the visible range in plot:
resolution = 100
X = vacc_list = np.linspace(minX, maxX, resolution)
Y = preval_list = np.linspace(minY,maxY, resolution)
Z = np.array([[Re(R0, v, p) for v in vacc_list] for p in preval_list]) # Y=preval, X=vacc



# r0 = 4.7
# res = np.linspace(0.05,0.95,numAreas)*r0
res = np.arange(0,R0+0.25,0.25)
numAreas = len(res)

cmap = LinearSegmentedColormap.from_list('unnamed', 
                                                    [
                                                        (0,1,0),
                                                        (1,0.9,0.7),
                                                        (1,0,0),
                                                        (1,0,0),
                                                        (1,0,0),
                                                        (1,0,0),
                                                    ] 
                                                    , N=numAreas)

ax1.contourf(X, Y, Z, levels=res,cmap=cmap)


# for i, re in enumerate([1]):
#     IsoVac = np.linspace(0,1-(re/R0),100)
#     ax1.plot(IsoVac, (lambda V: P(re, R0, V))(IsoVac), linewidth = 10,alpha=0.5,zorder=5, color="grey", label=int(i==0)*"$R_{e} = 1$-Isocline")

CS = ax1.contour(X, Y, Z, levels=res, zorder=6,colors='k') 
# clab_pos = [(1-np.sqrt(y),1-np.sqrt(y)) for y in res/R0]
clab_pos = [(1-np.sqrt(y),1-np.sqrt(y)) for y in res/R0]
# numLabelsToShow = 10
# clab_pos = list(zip(np.linspace(minX,maxX,numLabelsToShow),np.linspace(minY,maxY,numLabelsToShow)))
ax1.clabel(CS,fmt='%.2f', inline=True, fontsize=10,manual=clab_pos)
# ax.clabel(CS,fmt='%.2f', inline=True, fontsize=10)
    

ax1.set_xlim([minX,maxX])
ax1.set_ylim([minY,maxY])

plt.savefig(path_figs+'Vacc_Inci_Pars_semidependent')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# komToShow = ['København','Ishøj','Brøndby','Ballerup','Århus','Langeland','Høje Tåstrup','Glostrup','Frederiksberg']
# komToShow = ['Ishøj','Albertslund','Glostrup','Brøndby','Fredensborg','Tårnby','Dragør','Hvidovre','København','Frederiksberg']
komToShow = ['Ishøj','Albertslund','Glostrup','Brøndby','Fredensborg','Vesthimmerlands','Struer','Skive','Læsø']
# komToShow = kortdf.Kommunenavn.unique()

df_kommune_slice = df_kommune[df_kommune.Kommunenavn.isin(komToShow)]
df_kommune_slice.reset_index(inplace=True)


In [13]:
fig,ax1 = plt.subplots(figsize=(8,8))

minX = 0
maxX = 1
minY = 0
maxY = 1

minX = 0.5
maxX = 0.9
minY = 0
maxY = 0.3


R0 = 4.7
# R0 = 6.0

S = lambda V, Preval : (1-V)*(1-Preval)
S = lambda V, Preval : (1-V-Preval)
Re = lambda R0, V, Preval: R0*S(V, Preval)
V = lambda Re, R0, Preval: 1 - (Re/R0)*(1/(1-Preval))
P = lambda Re, R0, Vacc: 1 - (Re/R0)*(1/(1-Vacc))

# Adjust here, for the visible range in plot:
resolution = 100
X = vacc_list = np.linspace(minX, maxX, resolution)
Y = preval_list = np.linspace(minY,maxY, resolution)
Z = np.array([[Re(R0, v, p) for v in vacc_list] for p in preval_list]) # Y=preval, X=vacc



# r0 = 4.7
# res = np.linspace(0.05,0.95,numAreas)*r0
res = np.arange(0,R0+0.25,0.25)
numAreas = len(res)

cmap = LinearSegmentedColormap.from_list('unnamed', 
                                                    [
                                                        (0,1,0),
                                                        (1,0.9,0.7),
                                                        (1,0,0),
                                                        (1,0,0),
                                                        (1,0,0),
                                                        (1,0,0),
                                                    ] 
                                                    , N=numAreas)

ax1.contourf(X, Y, Z, levels=res,cmap=cmap)


# for i, re in enumerate([1]):
#     IsoVac = np.linspace(0,1-(re/R0),100)
#     ax1.plot(IsoVac, (lambda V: P(re, R0, V))(IsoVac), linewidth = 10,alpha=0.5,zorder=5, color="grey", label=int(i==0)*"$R_{e} = 1$-Isocline")

CS = ax1.contour(X, Y, Z, levels=res, zorder=6,colors='k') 
# clab_pos = [(1-np.sqrt(y),1-np.sqrt(y)) for y in res/R0]
clab_pos = [(1-np.sqrt(y),1-np.sqrt(y)) for y in res/R0]
numLabelsToShow = 10
clab_pos = list(zip(np.linspace(minX,maxX,numLabelsToShow),np.linspace(minY,maxY,numLabelsToShow)))
ax1.clabel(CS,fmt='%.2f', inline=True, fontsize=10,manual=clab_pos)
# ax.clabel(CS,fmt='%.2f', inline=True, fontsize=10)
    

# # ax1.plot(ratioVacc,numInf/popSize,'*',zorder=19)
# # ax1.plot(ratioVacc,2*numInf/popSize,'*r',zorder=19)
# for k in range(len(ratioVacc)):
#     # ax1.plot(ratioVacc[k],numInf[k]/popSize[k],'*',zorder=19)
#     ax1.plot([ratioVacc[k],ratioVacc[k]],[numInf[k]/popSize[k],2*numInf[k]/popSize[k]],'.-',zorder=19)
#     ax1.text(ratioVacc[k],numInf[k]/popSize[k],komNames[k],zorder=20)
    

for k in range(len(df_kommune_slice)):
    # ax1.plot(100*df_kommune.FaerdigvaccRatio[k],100*df_kommune.CasesSum[k]/df_kommune.Indbyggere[k],'k.')
    # ax1.text(100*df_kommune.FaerdigvaccRatio[k],100*df_kommune.CasesSum[k]/df_kommune.Indbyggere[k],df_kommune.Kommunenavn[k])
    ax1.plot(df_kommune_slice.FaerdigvaccRatio[k],df_kommune_slice.CasesSum[k]/df_kommune_slice.Indbyggere[k],'m.')
    ax1.text(df_kommune_slice.FaerdigvaccRatio[k],df_kommune_slice.CasesSum[k]/df_kommune_slice.Indbyggere[k],df_kommune_slice.Kommunenavn[k])
    
# ax.axis('equal')
ax1.set_xlim([minX,maxX])
ax1.set_ylim([minY,maxY])

plt.savefig(path_figs+'KommuneFigur_Dependent')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
fig,ax1 = plt.subplots(figsize=(8,8))

minX = 0
maxX = 1
minY = 0
maxY = 1

minX = 0.5
maxX = 0.9
minY = 0
maxY = 0.3


R0 = 4.7
# R0 = 6.0

S = lambda V, Preval : (1-V)*(1-Preval)
# S = lambda V, Preval : (1-V-Preval)
Re = lambda R0, V, Preval: R0*S(V, Preval)
V = lambda Re, R0, Preval: 1 - (Re/R0)*(1/(1-Preval))
P = lambda Re, R0, Vacc: 1 - (Re/R0)*(1/(1-Vacc))

# Adjust here, for the visible range in plot:
resolution = 100
X = vacc_list = np.linspace(minX, maxX, resolution)
Y = preval_list = np.linspace(minY,maxY, resolution)
Z = np.array([[Re(R0, v, p) for v in vacc_list] for p in preval_list]) # Y=preval, X=vacc



# r0 = 4.7
# res = np.linspace(0.05,0.95,numAreas)*r0
res = np.arange(0,R0+0.25,0.25)
numAreas = len(res)

cmap = LinearSegmentedColormap.from_list('unnamed', 
                                                    [
                                                        (0,1,0),
                                                        (1,0.9,0.7),
                                                        (1,0,0),
                                                        (1,0,0),
                                                        (1,0,0),
                                                        (1,0,0),
                                                    ] 
                                                    , N=numAreas)

ax1.contourf(X, Y, Z, levels=res,cmap=cmap)


# for i, re in enumerate([1]):
#     IsoVac = np.linspace(0,1-(re/R0),100)
#     ax1.plot(IsoVac, (lambda V: P(re, R0, V))(IsoVac), linewidth = 10,alpha=0.5,zorder=5, color="grey", label=int(i==0)*"$R_{e} = 1$-Isocline")

CS = ax1.contour(X, Y, Z, levels=res, zorder=6,colors='k') 
# clab_pos = [(1-np.sqrt(y),1-np.sqrt(y)) for y in res/R0]
clab_pos = [(1-np.sqrt(y),1-np.sqrt(y)) for y in res/R0]
numLabelsToShow = 10
clab_pos = list(zip(np.linspace(minX,maxX,numLabelsToShow),np.linspace(minY,maxY,numLabelsToShow)))
ax1.clabel(CS,fmt='%.2f', inline=True, fontsize=10,manual=clab_pos)
# ax.clabel(CS,fmt='%.2f', inline=True, fontsize=10)
    

# # ax1.plot(ratioVacc,numInf/popSize,'*',zorder=19)
# # ax1.plot(ratioVacc,2*numInf/popSize,'*r',zorder=19)
# for k in range(len(ratioVacc)):
#     # ax1.plot(ratioVacc[k],numInf[k]/popSize[k],'*',zorder=19)
#     ax1.plot([ratioVacc[k],ratioVacc[k]],[numInf[k]/popSize[k],2*numInf[k]/popSize[k]],'.-',zorder=19)
#     ax1.text(ratioVacc[k],numInf[k]/popSize[k],komNames[k],zorder=20)
    

for k in range(len(df_kommune_slice)):
    # ax1.plot(100*df_kommune.FaerdigvaccRatio[k],100*df_kommune.CasesSum[k]/df_kommune.Indbyggere[k],'k.')
    # ax1.text(100*df_kommune.FaerdigvaccRatio[k],100*df_kommune.CasesSum[k]/df_kommune.Indbyggere[k],df_kommune.Kommunenavn[k])
    ax1.plot(df_kommune_slice.FaerdigvaccRatio[k],df_kommune_slice.CasesSum[k]/df_kommune_slice.Indbyggere[k],'m.')
    ax1.text(df_kommune_slice.FaerdigvaccRatio[k],df_kommune_slice.CasesSum[k]/df_kommune_slice.Indbyggere[k],df_kommune_slice.Kommunenavn[k])
    
# ax.axis('equal')
ax1.set_xlim([minX,maxX])
ax1.set_ylim([minY,maxY])

plt.savefig(path_figs+'KommuneFigur_Independent')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
fig,ax1 = plt.subplots(figsize=(8,8))

minX = 0
maxX = 1
minY = 0
maxY = 1

minX = 0.5
maxX = 0.9
minY = 0
maxY = 0.3


R0 = 4.7
# R0 = 6.0

S = lambda V, Preval : (1-V)*(1-Preval)
S = lambda V, Preval : (1-V-Preval+0.5*V*Preval)
Re = lambda R0, V, Preval: R0*S(V, Preval)
V = lambda Re, R0, Preval: 1 - (Re/R0)*(1/(1-Preval))
P = lambda Re, R0, Vacc: 1 - (Re/R0)*(1/(1-Vacc))

# Adjust here, for the visible range in plot:
resolution = 100
X = vacc_list = np.linspace(minX, maxX, resolution)
Y = preval_list = np.linspace(minY,maxY, resolution)
Z = np.array([[Re(R0, v, p) for v in vacc_list] for p in preval_list]) # Y=preval, X=vacc



# r0 = 4.7
# res = np.linspace(0.05,0.95,numAreas)*r0
res = np.arange(0,R0+0.25,0.25)
numAreas = len(res)

cmap = LinearSegmentedColormap.from_list('unnamed', 
                                                    [
                                                        (0,1,0),
                                                        (1,0.9,0.7),
                                                        (1,0,0),
                                                        (1,0,0),
                                                        (1,0,0),
                                                        (1,0,0),
                                                    ] 
                                                    , N=numAreas)

ax1.contourf(X, Y, Z, levels=res,cmap=cmap)


# for i, re in enumerate([1]):
#     IsoVac = np.linspace(0,1-(re/R0),100)
#     ax1.plot(IsoVac, (lambda V: P(re, R0, V))(IsoVac), linewidth = 10,alpha=0.5,zorder=5, color="grey", label=int(i==0)*"$R_{e} = 1$-Isocline")

CS = ax1.contour(X, Y, Z, levels=res, zorder=6,colors='k') 
# clab_pos = [(1-np.sqrt(y),1-np.sqrt(y)) for y in res/R0]
# clab_pos = [(1-np.sqrt(y),1-np.sqrt(y)) for y in res/R0]
numLabelsToShow = 10
clab_pos = list(zip(np.linspace(minX,maxX,numLabelsToShow),np.linspace(minY,maxY,numLabelsToShow)))
ax1.clabel(CS,fmt='%.2f', inline=True, fontsize=10,manual=clab_pos)
# ax.clabel(CS,fmt='%.2f', inline=True, fontsize=10)
    

# # ax1.plot(ratioVacc,numInf/popSize,'*',zorder=19)
# # ax1.plot(ratioVacc,2*numInf/popSize,'*r',zorder=19)
# for k in range(len(ratioVacc)):
#     # ax1.plot(ratioVacc[k],numInf[k]/popSize[k],'*',zorder=19)
#     ax1.plot([ratioVacc[k],ratioVacc[k]],[numInf[k]/popSize[k],2*numInf[k]/popSize[k]],'.-',zorder=19)
#     ax1.text(ratioVacc[k],numInf[k]/popSize[k],komNames[k],zorder=20)
    

for k in range(len(df_kommune_slice)):
    # ax1.plot(100*df_kommune.FaerdigvaccRatio[k],100*df_kommune.CasesSum[k]/df_kommune.Indbyggere[k],'k.')
    # ax1.text(100*df_kommune.FaerdigvaccRatio[k],100*df_kommune.CasesSum[k]/df_kommune.Indbyggere[k],df_kommune.Kommunenavn[k])
    ax1.plot(df_kommune_slice.FaerdigvaccRatio[k],df_kommune_slice.CasesSum[k]/df_kommune_slice.Indbyggere[k],'m.')
    ax1.text(df_kommune_slice.FaerdigvaccRatio[k],df_kommune_slice.CasesSum[k]/df_kommune_slice.Indbyggere[k],df_kommune_slice.Kommunenavn[k])
    
# ax.axis('equal')
ax1.set_xlim([minX,maxX])
ax1.set_ylim([minY,maxY])

plt.savefig(path_figs+'KommuneFigur_semidependent')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
df_kommune_slice.Faerdigvacc

0    23496
1    16769
2    18422
3    14476
4    30650
5    16366
6    35542
7     1448
Name: Faerdigvacc, dtype: int64

In [17]:
# komToShow = ['København','Ishøj','Brøndby','Ballerup','Århus','Langeland','Høje Tåstrup','Frederiksberg']
komToShow = kortdf.Kommunenavn.unique()

kortdf_slice = kortdf[kortdf.Kommunenavn.isin(komToShow)]
idsToShow = kortdf_slice.Kommune.values
countdf_slice = countdf[countdf.Kommune.isin(idsToShow)]
vaccdf_slice = vaccdf[vaccdf.Kommunenavn.isin(komToShow)]
komNames = kortdf_slice.Kommunenavn.values

numInf = countdf_slice.groupby('Kommune')['Bekræftede tilfælde'].sum().values
ratioVacc = (vaccdf_slice['Vacc.dækning faerdigvacc. (%)']/100).values
popSize = vaccdf_slice['Antal borgere'].values